In [1]:
seed_value= 1

import os
os.environ['PYTHONHASHSEED']=str(seed_value)

import random
random.seed(seed_value)

import numpy as np
np.random.seed(seed_value)

import tensorflow as tf
tf.random.set_seed(seed_value)# for later versions: 
# tf.compat.v1.set_random_seed(seed_value)

from keras import backend as K
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder,MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from keras.layers import Input, Reshape, Embedding, Concatenate, Dropout, Dense
from keras.models import Model
from keras.callbacks import EarlyStopping

In [3]:
df = pd.read_csv('cleaned_data/data_with_features.csv')
df = df.drop(df.columns[0], axis = 1)

In [4]:
#add right target
df_lst = []

for user in np.unique(df['EthicaID']): 
    df_single_user = df.loc[df['EthicaID'] == user]     
    real_target = df_single_user[['target']][1:len(df_single_user)].rename({'target': 'real_target'}, axis = 1)
    df_single_user = pd.concat([df_single_user.reset_index(drop = True), real_target.reset_index(drop = True)], axis =1)    
    df_lst.append(df_single_user)

In [5]:
df_full = pd.concat(df_lst, axis = 0)

In [6]:
df_full.tail()

,EthicaID,use_duration,duration_ongoing_session,time_to_next_app,time_to_next_app_t+1,age_category,sex,notification,battery,category,t+1 category,hours,weekday,geohash,location_cluster,target,real_target
7277,User #25569,-0.237135,0.927877,-0.060989,-0.060731,"17.749, 19.25",1,False,17.0,Social,Social,1,2,u15uq,1,medium,long
7278,User #25569,5.602708,0.935795,-0.060730,0.035432,"17.749, 19.25",1,False,17.0,Social,Communication,1,2,u15uq,1,long,long
7279,User #25569,0.743716,-0.575649,0.035435,-0.061103,"17.749, 19.25",1,False,27.0,Communication,Productivity,1,2,u15uq,1,long,long
7280,User #25569,-0.148606,-0.441971,-0.061102,-0.061125,"17.749, 19.25",1,False,27.0,Productivity,Communication,1,2,u15uq,1,long,long
7281,User #25569,0.830987,-0.423806,-0.061124,NaN,"17.749, 19.25",1,False,27.0,Communication,NaN,1,2,u15uq,1,long,NaN


In [7]:
df_full.isna().sum()

EthicaID                      0
use_duration                  0
duration_ongoing_session      0
time_to_next_app              0
time_to_next_app_t+1        186
age_category                  0
sex                           0
notification                  0
battery                       0
category                      0
t+1 category                186
hours                         0
weekday                       0
geohash                       0
location_cluster              0
target                        0
real_target                 186
dtype: int64

In [8]:
df_full = df_full.dropna()

In [9]:
df_full.columns

Index(['EthicaID', 'use_duration', 'duration_ongoing_session',
       'time_to_next_app', 'time_to_next_app_t+1', 'age_category', 'sex',
       'notification', 'battery', 'category', 't+1 category', 'hours',
       'weekday', 'geohash', 'location_cluster', 'target', 'real_target'],
      dtype='object')

In [11]:
#select feature subset
df_model = df_full[['use_duration', 'duration_ongoing_session',
       'time_to_next_app', 'age_category', 'sex',
       'notification', 'battery', 'category', 'hours',
       'weekday', 'geohash', 'location_cluster', 'target']]

In [12]:
cat_col = ['age_category', 'sex', 'notification', 'battery', 'category', 'hours',
           'weekday', 'geohash', 'location_cluster', 'target']
num_col_lst =  ['use_duration', 'duration_ongoing_session', 'time_to_next_app']
target = 'real target'

In [13]:
#make encoders
encoders = {}  
for v in cat_col:  
    le = LabelEncoder()
    le.fit(df_model[v].values)
    encoders[v] = le
    df_model.loc[:, v] = le.transform(df_model[v].values)
    print('{0}: {1}'.format(v, le.classes_))

C:\Users\katha\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


age_category: ['17.749, 19.25' '19.25, 20.083' '20.083, 21.75' '21.75, 41.333']
sex: [1 2]
notification: [False  True]
battery: [  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.
  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.
  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.
  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.
  98.  99. 100.]
category: ['Art & Design' 'Auto & Vehicles' 'Beauty' 'Books & Reference' 'Business'
 'Comics' 'Communication' 'Dating' 'Education' 'Entertainment' 'Events'
 'Finance' 'Food & Drink' 'Games' 'Health & Fitness' 'House & Home'
 'Lifestyle' 'Maps & Navigation' 'Medical' 'Music & Audio'
 'News & Magazines' 'Parenting' 'Personalization' 'Photography'
 'Productiv

In [14]:
#test train split
X_train, X_test, y_train, y_test = train_test_split(df_model, df_full['real_target'], test_size=0.2, random_state=1)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=1)

In [17]:
X_train = X_train[cat_col + num_col_lst].copy()  
X_val = X_valid[cat_col + num_col_lst].copy()
X_test = X_test[cat_col + num_col_lst].copy()

In [18]:
y = y_train.astype(str).values
y_val = y_valid.astype(str).values
y_test = y_test.astype(str).values
y = y.reshape(len(y), 1)
y_val = y_val.reshape(len(y_val), 1)
y_test = y_test.reshape(len(y_test), 1)

In [19]:
def prepare_targets(y_train, y_valid, y_test):
    le=MultiLabelBinarizer()
    le.fit(y_train)
    y_train_enc = le.transform(y_train)
    y_valid_enc = le.transform(y_valid)
    y_test_enc = le.transform(y_test)
    return y_train_enc, y_valid_enc, y_test_enc

In [20]:
#encode targets
y, y_val, y_test =  prepare_targets(y, y_val, y_test)

In [21]:
#prepare features for transformation
for v in cat_col:  
    X_train[v] = X_train[v].astype('int').astype('category').cat.as_ordered()
    X_val[v] = X_val[v].astype('int').astype('category').cat.as_ordered()
    X_test[v] = X_test[v].astype('int').astype('category').cat.as_ordered()
for v in num_col_lst:  
    X_train[v] = X_train[v].astype('float32')
    X_val[v] = X_val[v].astype('float32')
    X_test[v] = X_test[v].astype('float32')

In [22]:
df_cat = df_model[['age_category', 'sex', 'notification', 'battery', 'category', 'hours',
           'weekday', 'geohash', 'location_cluster', 'target']].astype(('category'))

In [23]:
#define embeddings size
cat_sizes = [(c, len(df_cat[c].cat.categories)) for c in cat_col]  
cat_sizes
embedding_sizes = [(c, min(50, (c + 1) // 2)) for _, c in cat_sizes]  
embedding_sizes

[('age_category', 4),
 ('sex', 2),
 ('notification', 2),
 ('battery', 101),
 ('category', 32),
 ('hours', 24),
 ('weekday', 7),
 ('geohash', 1969),
 ('location_cluster', 8),
 ('target', 3)]

In [25]:
X_array = []  
X_val_array = []
X_test_array = []

for i, v in enumerate(cat_col):  
    X_array.append(X_train.iloc[:, i])
    X_val_array.append(X_val.iloc[:, i])
    X_test_array.append(X_test.iloc[:, i])

X_array.append(X_train.iloc[:, len(cat_col):])  
X_val_array.append(X_val.iloc[:, len(cat_col):])
X_test_array.append(X_test.iloc[:, len(cat_col):])


len(X_array), len(X_val_array), len(X_test_array)

(11, 11, 11)

In [26]:
#define model structure
def fit_model(neurons):
    inputs = []
    embed_layers = []
    for (c, (in_size, out_size)) in zip(cat_col, embedding_sizes):
        i = Input(shape=(1, )) # dim hinzufügen
        o = Embedding(in_size, out_size, name=c)(i)
        o = Reshape(target_shape=(out_size,))(o)
        inputs.append(i)
        embed_layers.append(o)
    embed = Concatenate()(embed_layers)
    cont_input = Input(shape=(len(num_col_lst),))
    inputs.append(cont_input)
    x = Concatenate()([embed, cont_input])
    dense = Dense(neurons, activation= 'sigmoid', kernel_initializer= 'he_normal')(x)
    d = Dropout(0.2)(dense)

    output = Dense(3, activation= 'softmax')(d)
    model = Model(inputs=inputs, outputs=output)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [27]:
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience = 5, restore_best_weights=True)

In [28]:
import matplotlib.pyplot as plt

In [29]:
#hyperparameter optimization loop
from datetime import datetime
start_time = datetime.now()
for num_neurons in [256]: 
    for batch_size in [256]:
        model = fit_model(num_neurons)
        history = model.fit(X_array, y, epochs=50, validation_data = (X_val_array, y_val), callbacks  = [es], verbose = 1, 
                           batch_size = batch_size)
        _, accuracy = model.evaluate(X_val_array, y_val, verbose=1)
        with open('model/Baseline/Baseline_all_features_excl_special+Dropout/Baseline_all_features_excl_special+Dropout.txt', 'a') as fp:
            fp.writelines(['Number of Neurons: ' + str(num_neurons) +
                           '\n', 'Batch Size: ' + str(batch_size) + '\n', 'seed1\n',
                           'Accuracy: ' + str(accuracy) + '\n\n'])
        model.save('model/Baseline/Baseline_all_features_excl_special+Dropout/model_neurons_'+ str(num_neurons) + 'batch_size' +
                   str(batch_size)+ '_seed1.h5')
        plt.figure(figsize=(10, 6))
        plt.plot(history.history['accuracy'], color = 'dodgerblue')
        plt.plot(history.history['val_accuracy'], color = 'rebeccapurple')
        plt.title('model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.xticks(range(0, len(history.history['accuracy'])+1, 2))
        plt.legend(['train', 'valid'], loc='upper left')
        plt.savefig('model/Baseline/Baseline_all_features_excl_special+Dropout/model_neurons_'+ str(num_neurons) + #
                    'batch_size' + str(batch_size)+ '_seed1.png')
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Epoch 1/50
13812/13812 [==============================] - 117s 8ms/step - loss: 1.0823 - accuracy: 0.3911 - val_loss: 1.0782 - val_accuracy: 0.3968
Epoch 2/50
13812/13812 [==============================] - 95s 7ms/step - loss: 1.0770 - accuracy: 0.3988 - val_loss: 1.0747 - val_accuracy: 0.4021
Epoch 3/50
13812/13812 [==============================] - 99s 7ms/step - loss: 1.0741 - accuracy: 0.4033 - val_loss: 1.0721 - val_accuracy: 0.4062
Epoch 4/50
13812/13812 [==============================] - 102s 7ms/step - loss: 1.0723 - accuracy: 0.4060 - val_loss: 1.0712 - val_accuracy: 0.4076
Epoch 5/50
13812/13812 [==============================] - 104s 8ms/step - loss: 1.0711 - accuracy: 0.4078 - val_loss: 1.0708 - val_accuracy: 0.4090
Epoch 6/50
13812/13812 [==============================] - 102s 7ms/step - loss: 1.0701 - accuracy: 0.4094 - val_loss: 1.0701 - val_accuracy: 0.4108
Epoch 7/50
13812/13812 [==============================] - 118s 9ms/step - loss: 1.0693 - accuracy: 0.4108 - val_lo

In [30]:
import keras

In [31]:
#insert best hyperparameters to optain predictive accuracy
model = keras.models.load_model('model/Baseline/Baseline_all_features_excl_special+Dropout/model_neurons_256batch_size256_seed1.h5')
_, accuracy_test = model.evaluate(X_test_array, y_test, verbose=1)
print(accuracy_test)

_, accuracy_valid = model.evaluate(X_val_array, y_val, verbose=1)
print(accuracy_valid)

_, accuracy_train = model.evaluate(X_array, y, verbose=1)
print(accuracy_train)

with open('model/Baseline/Baseline_all_features_excl_special+Dropout/Baseline_all_features_excl_special+Dropout.txt', 'a') as fp:
    fp.writelines(['Best model accuracy on test set: ' + str(accuracy_test) + '\n\n', 
                   'Best model accuracy on valid set: ' + str(accuracy_valid) + '\n\n', 
                   'Best model accuracy on train set: ' + str(accuracy_train) + '\n\n'])

13812/13812 [==============================] - 24s 2ms/step - loss: 1.0703 - accuracy: 0.4105
0.41053998470306396
13812/13812 [==============================] - 24s 2ms/step - loss: 1.0700 - accuracy: 0.4125
0.41252294182777405
110494/110494 [==============================] - 202s 2ms/step - loss: 1.0588 - accuracy: 0.4238
0.42378199100494385


In [32]:
#wirte training time to file
with open('model/Baseline/Baseline_all_features_excl_special+Dropout/Baseline_all_features_excl_special+Dropout.txt', 'a') as fp:
    fp.writelines(['Time needed for training:', str(end_time - start_time)])